In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import sys, os

cd = os.path.split(os.getcwd())[0]
if cd not in sys.path:
    sys.path.append(cd)

from lib import noaa, bexarcrime

In [ ]:
dfw = noaa.noaa_from_web_small() # weather database handle

In [ ]:
dfc = bexarcrime.crime_from_web() # crime database handle

In [ ]:
dfc = bexarcrime.get_violent_crime(bexarcrime.crime_from_web())
dfc

Drop all useless columns from the crime dataframe

In [ ]:
dfc = dfc.drop(['CASE-CAUSE-NBR', 'FULL-NAME', 'ADDR-CITY', 'ADDR-STATE','ADDR-ZIP-CODE','OFFENSE-DESC','OFFENSE-TYPE',
               'REDUCED-OFFENSE-CODE','REDUCED-OFFENSE-DESC','REDUCED-OFFENSE-TYPE','LOCATION','CUSTODY-DATE','BOND-DATE',
               'BOND-STATUS','BOND-AMOUNT'],axis = 1)

Replace all values used to represent NULL

In [ ]:
dfw['Temperature'] = dfw['Temperature'].replace(9999,np.nan)
dfw['Pressure'] = dfw['Pressure'].replace(99999,np.nan)
dfw['Humidity'] = dfw['Humidity'].replace(999, np.nan)
dfw['Sky'] = dfw['Sky'].replace([9,99], np.nan)

Group the weather data by the average for a day

In [ ]:
dfw.groupby([dfw.Date.dt.year, dfw.Date.dt.month, dfw.Date.dt.day]).mean()